In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import random

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [2]:
import yfinance as yf
import requests
import json

In [3]:
s = '2022-01-04'
e = '2024-04-22'

download = False

In [4]:
df = pd.read_csv("https://query.data.world/s/pzd7uazpmpyortmg6fsuxiaoizllgu?dws=00000", encoding='ISO-8859-1')
display(df.head(10))

,Public Company Name (As Listed on Exchange),B Corp Impact Data ID,Ticker 1,Ticker 2,Ticker 3,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
0,"Allbirds, Inc.",001C000001SGhnyIAD,NASDAQGS:BIRD,NaN,NaN,2021.0,NaN,2016.0,active,NaN
1,Amalgamated Financial,001C000001K16JSIAZ,NASDAQGM:AMAL,NaN,NaN,2018.0,NaN,2017.0,active,NaN
2,"AppHarvest, Inc.",0013b00001o41JFAAY,NASDAQGS:APPH,NaN,NaN,2020.0,NaN,2019.0,delisted,2023.0
3,Arezzo&Co,0013b00001qHFddAAG,BOVESPA:ARZZ3,NaN,NaN,2011.0,NaN,2022.0,active,NaN
4,Arowana International,001C000001ZzPK4IAN,ASX: AWN,NaN,NaN,2013.0,2021.0,2018.0,delisted,NaN
5,Australian Ethical Investment Ltd.,001C0000014QBrxIAG,ASX:AEF,NaN,NaN,2002.0,NaN,2014.0,active,NaN
6,Benefit Systems S.A.,001C000001TZiM4IAL,WSE:BFT,NaN,NaN,2013.0,NaN,2018.0,active,NaN
7,Biome Australia,0013b00001vKRdWAAW,ASX:BIO,NaN,NaN,2021.0,NaN,2021.0,active,NaN
8,Cafe Direct,001C000001a2QxxIAE,ETHEX,NaN,NaN,NaN,NaN,2018.0,active,NaN
9,"Charlotte's Web Holdings, Inc.",001C000001XtSSRIA3,TSX:CWEB,OTCPK:CWBH.F,NaN,2018.0,NaN,2020.0,active,NaN


In [5]:
df.drop(['B Corp Impact Data ID', 'Ticker 2', 'Ticker 3'], axis=1, inplace = True)
print(len(df))
df.dropna(axis=0, subset=['Public Company Name (As Listed on Exchange)'], inplace=True)
print(len(df))

999
68


In [6]:
display(df[df['Public Listing Status'] == 'delisted'])

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified
2,"AppHarvest, Inc.",NASDAQGS:APPH,2020.0,NaN,2019.0,delisted,2023.0
4,Arowana International,ASX: AWN,2013.0,2021.0,2018.0,delisted,NaN
14,Freshlocal Solutions,TSE: LOCL,2019.0,2021.0,2013.0,delisted,NaN


In [7]:
temp = df.copy()
temp = temp[temp['Public Listing Status'] == 'active']

temp['Ticker 1'].replace(": ", ":", regex=True, inplace=True)


print(len(temp))
temp['Exchange'] = temp['Ticker 1'].str.split(":").str[0]
temp['Ticker2'] = temp['Ticker 1'].str.split(":").str[1]
temp.dropna(subset=["Ticker2"], inplace=True) # Remove any NaN tickers

display(temp.head())

65


,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2
0,"Allbirds, Inc.",NASDAQGS:BIRD,2021.0,NaN,2016.0,active,NaN,NASDAQGS,BIRD
1,Amalgamated Financial,NASDAQGM:AMAL,2018.0,NaN,2017.0,active,NaN,NASDAQGM,AMAL
3,Arezzo&Co,BOVESPA:ARZZ3,2011.0,NaN,2022.0,active,NaN,BOVESPA,ARZZ3
5,Australian Ethical Investment Ltd.,ASX:AEF,2002.0,NaN,2014.0,active,NaN,ASX,AEF
6,Benefit Systems S.A.,WSE:BFT,2013.0,NaN,2018.0,active,NaN,WSE,BFT


In [8]:
display(temp.loc[40:,:])

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2
41,Urb-it AB (publ),OM:URBIT,2017.0,NaN,2020.0,active,NaN,OM,URBIT
42,Vantea SMART S.p.A.,BIT:VNT,2021.0,NaN,2022.0,active,NaN,BIT,VNT
43,Viña Concha y Toro S.A.,SNSE:CONCHATORO,NaN,NaN,2021.0,active,NaN,SNSE,CONCHATORO
44,"Vital Farms, Inc.",NASDAQGM:VITL,2020.0,NaN,2015.0,active,NaN,NASDAQGM,VITL
45,VivoPower International PLC,NASDAQCM:VVPR,2016.0,NaN,2018.0,active,NaN,NASDAQCM,VVPR
46,Volpara Health Technologies Limited,ASX:VHT,2016.0,NaN,2022.0,active,NaN,ASX,VHT
47,Warby Parker Inc.,NYSE:WRBY,2021.0,NaN,2011.0,active,NaN,NYSE,WRBY
48,Weleda AG Group,OTC-X:BEKB:Weleda N-PS,NaN,NaN,2021.0,active,NaN,OTC-X,BEKB
49,Zevia PBC,NYSE:ZVIA,2021.0,NaN,2021.0,active,NaN,NYSE,ZVIA
51,Aussie Broadband Ltd,ASX:ABB,2020.0,NaN,2023.0,active,NaN,ASX,ABB


### Stocks are from Global Exchanges - build and maintain an exchange conversion dictionary to interface with Yahoo Finance

In [9]:
exchange_dict = {'ASX': '.AX', 
                 'WSE': '.WA',
                 'BOVESPA': '.SA',
                 'ETHEX': '',
                 'TSX': '.TO',
                 'ENXTPA': '.PA',
                 'BME': '.MC',
                 'ENXTBR': '.BR',
                 'KLSE': '.KL',
                 'LSE': '.L',
                 'TWSE': '.TW',
                 'NZSE': '.NZ',
                 'SGX': '.SI',
                 'BIT': '.MI',
                 'TASE': '.TA',
                 'XMIL': '.MI',
                 'NZE': '.NZ',
                 'BVMF': '.SA',
                 'HKG': '.HK',
                 'LON': '.L',
                 'MIL': '.MI',
                 'MOVI3': '.SA',
                 'TSE': '.T',
                 'BRK': '.L',
                 'SNSE': '.SN',
                 'MOVIDA B3': '.SA',
                 'NYSE': '',
                 'NASDAQGS': '',
                 'NASDAQGM': '',
                 'NASDAQCM': ''
                }

exchange_map = {exchanges: extention
                 for exchanges, extention in exchange_dict.items()
                 for exchange in exchanges}

temp['Intl Extention'] = temp['Exchange'].map(exchange_map)

In [10]:
temp['Ticker'] = temp['Ticker2'] + temp['Intl Extention'] 
mask = temp['Ticker'].isna()

# Check to see if we missed any:
temp[mask]

,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
19,Heidelberg District Community Enterprise Limited,NSX:EIC,2019.0,NaN,2019.0,active,NaN,NSX,EIC,NaN,NaN
36,SIDERPERU,BVL:SIDERC1,2006.0,NaN,2022.0,active,NaN,BVL,SIDERC1,NaN,NaN
41,Urb-it AB (publ),OM:URBIT,2017.0,NaN,2020.0,active,NaN,OM,URBIT,NaN,NaN
48,Weleda AG Group,OTC-X:BEKB:Weleda N-PS,NaN,NaN,2021.0,active,NaN,OTC-X,BEKB,NaN,NaN
56,Vinedos Emiliana S.A.,XSGO:EMILIANA,1996.0,NaN,2023.0,active,NaN,XSGO,EMILIANA,NaN,NaN


In [11]:
#Find datafeed issues
tickers = temp['Ticker'].to_list()

count = 0
for t in tickers:
    try:
        ticker = yf.Ticker(t)
        info = None
    except:
        count +=1
        print(f"Cannot get info of {t}, it probably does not exist")

print("Total nans =", count, "which is", round(count/temp.shape[0],2)*100, '% of the total')


Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Cannot get info of nan, it probably does not exist
Total nans = 5 which is 8.0 % of the total


In [12]:
# Remove Nans from stock universe:

b_corp_df = temp[-mask].copy()
display(b_corp_df.head())


,Public Company Name (As Listed on Exchange),Ticker 1,IPO Year,Year Delisted,Year Certified,Public Listing Status,Year Decertified,Exchange,Ticker2,Intl Extention,Ticker
0,"Allbirds, Inc.",NASDAQGS:BIRD,2021.0,NaN,2016.0,active,NaN,NASDAQGS,BIRD,,BIRD
1,Amalgamated Financial,NASDAQGM:AMAL,2018.0,NaN,2017.0,active,NaN,NASDAQGM,AMAL,,AMAL
3,Arezzo&Co,BOVESPA:ARZZ3,2011.0,NaN,2022.0,active,NaN,BOVESPA,ARZZ3,.SA,ARZZ3.SA
5,Australian Ethical Investment Ltd.,ASX:AEF,2002.0,NaN,2014.0,active,NaN,ASX,AEF,.AX,AEF.AX
6,Benefit Systems S.A.,WSE:BFT,2013.0,NaN,2018.0,active,NaN,WSE,BFT,.WA,BFT.WA


In [13]:
isolate_us = False
if isolate_us:

    exchanges = set(temp['Exchange'])
    print(exchanges)
    us_ex = ['NYSE', 'NASDAQGS','NASDAQGM', 'NASDAQCM']
    us = temp[temp['Exchange'].isin(us_ex)]
    print("Isolating US Companies")
    display(us.head())


### List of Companies with B-Corp Subsidiaries
Unilever (NYSE: UN): (Pukka (UK), Mãe Terra (Brazil), Sir Kensington’s (US), Ben & Jerry’s (US), Olly Nutrition (US), Seventh Generation (US), Sundial (US), T2 Tea (Australia), Graze (UK))

B2W (B3: BTOW3): Courrieros (Brazil)

Grupo Soma (B3: SOMA3): Companhia Hering (Brazil)

Arezzo&Co (B3: ARZZ3): Reserva (Brazil)

Procter & Gamble (NYSE: PG): (New Chapter (US))

AB Inbev: (4 Pines Brewing Company)

Banco Estado: (Banco Estado Microempresas, Caja Vecina)

Coca-Cola: (Innocent Drinks)

Fairfax Financial: (The Redwoods Group)

Gap: (Athleta, Hill City)

Kikkoman: (Country Life)

Lactalis: (Stonyfield Farm)

Land O’ Lakes: (Vermonta Creamery)

Nestle: (Essential Living Foods, Garden of Life, Lily's Kitchen)

Oppenheimer Funds Inc.: (SNW Asset Management)

Rakuten: (OverDrive)

The Hain Celestial Group: (Ella’s Kitchen)

Vina Concha y Toro: (Fetzer Vineyards)


Hortfruti S.A.: Hortifruti Chile

Danone SA: (Danone North America, Alpro, Danone Waters of America, Danone Canada, -Aguas Danone Argentina, Bledina, Danone Aqua Indonesia, Danone Egypt, Danone Iberia, Danone Manifesto Ventures, Danone UK, Grameen Danone Foods, Happy Family, Les 2 Vaches, Danone Waters Germany, Danone Dairy Ireland, Danone Waters Spain, Danone Netherlands, Danone Dairy Belgium, Nutricia Bago, Danone ELN Greater China, Danone Japan, Volvic, Danone Brazil)

Caixabank (Imagin Ban)

Banco Sabadell (Sogeviso)

iSun (SunCommon)

In [14]:
add_subs = False
public_subs = ['UL', 'SOMA3.SA', 'ABI.BR', 'PG', 'FFH.TO', 'KO', 'GPS', 'KIK.F', 'NESN.SW',
               'OPY', '4755.T', 'HAIN', 'DANOY', 'CABK.MC', 'ISUN']

In [15]:
def get_stock_data (df):
    tickers_list = df['Ticker'].tolist()
    if add_subs:
        tickers_list = tickers_list + public_subs
    tickers_data= {}
    
    for ticker in tickers_list:
        ticker_object = yf.Ticker(ticker)
        
        #convert info() output from dictionary to dataframe
        temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
        temp.reset_index(inplace=True)
        temp.columns = ["Attribute", "Recent"]

        # add (ticker, dataframe) to main dictionary
        tickers_data[ticker] = temp

    return tickers_data
        
        

In [16]:
if isolate_us:
    load_stock_data = get_stock_data (us)
else:
    load_stock_data = get_stock_data (b_corp_df)
    
print("Number of Stock Records Loaded", len(load_stock_data))

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SZL.AX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SZL.AX&crumb=I36BWyifig.


Number of Stock Records Loaded 57


In [17]:
# Clean df:
stock_data = pd.concat(load_stock_data) #turn from dict to df
stock_data = stock_data.reset_index()

del stock_data["level_1"] # clean up unnecessary column
stock_data.columns = ["Ticker", "Attribute", "Recent"] # update column names


attr_to_keep = ['shortName', 'longName', 'industry', 'sector','previousClose', 'sharesOutstanding', 'impliedSharesOutstanding',
               'marketCap',  'currency', 'country', 'website', 'enterpriseValue',
                'trailingEps', 'forwardEps', 'enterprisetoEbitda', 'ebitda', 'grossMargins', 'ebitdaMargins', 'operatingMargins']


stock_data = stock_data[stock_data["Attribute"].isin(attr_to_keep)].reset_index()
del stock_data["index"] # clean up unnecessary column



display(stock_data.head(15))

,Ticker,Attribute,Recent
0,BIRD,country,United States
1,BIRD,website,https://www.allbirds.com
2,BIRD,industry,Apparel Retail
3,BIRD,sector,Consumer Cyclical
4,BIRD,previousClose,0.6116
5,BIRD,marketCap,99362312
6,BIRD,currency,USD
7,BIRD,enterpriseValue,58830720
8,BIRD,sharesOutstanding,103291000
9,BIRD,impliedSharesOutstanding,155838000


In [18]:
print("Number of Stocks:", len(stock_data['Ticker'].unique()))
display(stock_data['Ticker'].unique())

Number of Stocks: 54


array(['BIRD', 'AMAL', 'ARZZ3.SA', 'AEF.AX', 'BFT.WA', 'BIO.AX',
       'CWEB.TO', 'CLSA3.SA', 'MLMFI.PA', 'COUR', 'FLOW.TO', 'FREY.PA',
       'GFI.NZ', 'GROV', 'HAN.MC', 'HLZ.MC', 'INCLU.BR', 'IBAB.BR',
       'KPG.AX', 'KCT.L', 'KMD.NZ', 'LMND', 'LFG.AX', 'MOVI3.SA',
       'NTCO3.SA', '2897.TW', '1H2.SI', 'ALQP.PA', 'RETI.MI', '6088.T',
       'SML.NZ', 'COCO', 'VNT.MI', 'CONCHATORO.SN', 'VITL', 'VVPR',
       'VHT.AX', 'WRBY', 'ZVIA', 'ABB.AX', 'TIME.MI', 'CVT.NZ', 'DOX.MI',
       'SOMA3.SA', '1112.HK', 'HMGS.TA', '1346.HK', '0973.HK', 'MME.AX',
       'MPR.AX', 'TENG.L', 'TPX.L', 'XSG.L', 'XHS.MI'], dtype=object)

In [19]:
stock_data_filtered = stock_data.pivot(columns = 'Attribute', index='Ticker', values='Recent')
stock_data_filtered.columns

# More info:
#stock_data_filtered = stock_data_filtered[['shortName', 'longName','country', 'currency', 'previousClose',
#                                           'sharesOutstanding', 'impliedSharesOutstanding','marketCap', 'enterpriseValue',
#                                           'ebitda', 'forwardEps',  'trailingEps', 'grossMargins', 'operatingMargins', 'ebitdaMargins',
#                                             'industry', 'sector', 'website']]
# Only essential info:
stock_data_filtered = stock_data_filtered[['shortName', 'longName','country', 'currency', 'previousClose',
                                           'impliedSharesOutstanding','marketCap','industry', 'sector', 'website']]

display(stock_data_filtered.head(10))

Attribute,shortName,longName,country,currency,previousClose,impliedSharesOutstanding,marketCap,industry,sector,website
Ticker,,,,,,,,,,
0973.HK,L'OCCITANE,L'Occitane International S.A.,Luxembourg,HKD,32.25,1472169984,47992737792,Household & Personal Products,Consumer Defensive,https://group.loccitane.com
1112.HK,H&H INTL HLDG,Health and Happiness (H&H) International Holdi...,Hong Kong,HKD,9.94,645561024,6688011776,Packaged Foods,Consumer Defensive,https://www.hh.global
1346.HK,LEVER STYLE,Lever Style Corporation,Hong Kong,HKD,0.89,645899008,549626048,Apparel Manufacturing,Consumer Cyclical,https://www.leverstyle.com
1H2.SI,$ Olive Tree,Olive Tree Estates Limited,Singapore,SGD,0.08,115845000,9267600,Real Estate - Diversified,Real Estate,https://www.olivetreeestates.com
2897.TW,O-BANK CO LTD,"O-Bank Co., Ltd.",Taiwan,TWD,10.0,2716470016,27572168704,Banks - Regional,Financial Services,https://www.o-bank.com
6088.T,SIGMAXYZ HOLDINGS INC,SIGMAXYZ Holdings Inc.,Japan,JPY,1547.0,43137600,65001107456,Specialty Business Services,Industrials,https://www.sigmaxyz.com
ABB.AX,AUSSIEBAND FPO [ABB],Aussie Broadband Limited,Australia,AUD,3.73,297591008,1151221504,Telecom Services,Communication Services,https://www.aussiebroadband.com.au
AEF.AX,A ETHICAL FPO [AEF],Australian Ethical Investment Limited,Australia,AUD,4.54,112782000,517669408,Asset Management,Financial Services,https://www.australianethical.com.au
ALQP.PA,QUADPACK,"Quadpack Industries, S.A.",Spain,EUR,13.7,4382130,62375040,Packaging & Containers,Consumer Cyclical,https://www.quadpack.com


## Add in stock price as of beginning of index:

In [20]:
def get_starting_price(df, start = s):
    tickers_list = df.index.tolist()
    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    
    prices = pd.DataFrame.from_dict(first_object.history(start=start, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]

    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        try:
            temp = pd.DataFrame.from_dict(ticker_object.history(start=start, interval = "1d"))
            temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
            temp.columns = [ticker]
            prices = prices.merge(temp, how="outer", left_index=True, right_index=True)
        except:
            continue
    
    
    # Collapse date across international zones:
    prices.index = pd.to_datetime(prices.index).date
      
        
    prices = prices.groupby(prices.index).ffill() 
    prices = prices.groupby(prices.index).bfill()
    prices = prices.groupby(prices.index).last()
    
    
    return prices

In [21]:
data = get_starting_price(stock_data_filtered, start=s)
starting_px = data.filter(like='2022-01-04', axis=0).T

### Incorporate exchange rates to translate price and market cap into USD

In [22]:
# Get avaialable currencies
currency_dict = {}

url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/symbols"

headers = {}
try:
    with open('code.json') as file:
        headers = json.load(file)
except FileNotFoundError:
    print("Couldn't load credentials")

response = requests.get(url, headers=headers)

api_data = response.json()

    
for symbol, curr in api_data['symbols'].items():
    currency_dict[symbol] = [curr, 0]



Couldn't load credentials


KeyError: 'symbols'

In [ ]:
## Access Exchange Rates at Start of Measurement:
def get_rates(date):
    url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/" + str(date)

    querystring = {'base':'USD'}

    headers = {}
    try:
        with open('code.json') as file:
            headers = json.load(file)
    except FileNotFoundError:
        print("Couldn't load credentials")

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()


In [ ]:
## Combine date from exch_data with names / symbols in currency_dict:
exch_data = get_rates(date='2024-01-02')

for symbol, rate in exch_data['rates'].items():
    for key in currency_dict.keys():
        if symbol == key:
            currency_dict[key][1] = rate

In [ ]:
currency_dict

In [ ]:
## Add exhange rates and calculate USD equivalents:

#exchange_map = {exchanges: extention
#                 for exchanges, extention in exchange_dict.items()
#                 for exchange in exchanges}


currency_map = {symbols: rate
                 for symbols, name in currency_dict.items()
                 for rate in name}



stock_data_filtered['beginningUSDExchangeRate'] = stock_data_filtered['currency'].map(currency_map)


In [ ]:
stock_data_filtered = stock_data_filtered.join(starting_px)

In [ ]:
# Check to see that exchange rates mapped correctly:
stock_data_filtered.head(15)

In [ ]:
# Calculate USD price and USD marketcap
s_d = dt.strptime(s, '%Y-%M-%d').date()
stock_data_filtered['beginningCloseUSD'] = stock_data_filtered[s_d] / stock_data_filtered['beginningUSDExchangeRate']
stock_data_filtered['beginningMarketCapUSD'] = stock_data_filtered['impliedSharesOutstanding']* stock_data_filtered['beginningCloseUSD']

stock_data_filtered.head(20)

In [ ]:
if download:
    stock_data_filtered.to_csv("mkt_cap_details.csv")

## Filter Universe of Stocks by USD MarketCap

In [ ]:
def filter_index(df, size=True, num=True, size_thresh = 1000000000, num_thresh = 20, col='beginningMarketCapUSD'):
    temp = df.copy()
    if size == True:
        temp = temp[temp[col] >= size_thresh]
    
    if num == True:
        temp.sort_values(by=[col], ascending=False, inplace = True)
        temp = temp.iloc[0:num_thresh,:]
            
    if 'ABI.BR' in temp.index:
        temp.loc['ABI.BR','impliedSharesOutstanding'] = 1982790000
    
    return temp



In [ ]:
stock_index_top20 = filter_index(stock_data_filtered, size = False, num_thresh=20)
stock_index_top30 = filter_index(stock_data_filtered, size = False, num_thresh=30)
stock_index_over1b = filter_index(stock_data_filtered, size = True, num=False, size_thresh=1000000000)
stock_index_cutoff = filter_index(stock_data_filtered, size = False, num_thresh=40)

df_list = [stock_index_top20, stock_index_top30, stock_index_over1b, stock_index_cutoff]

print(stock_index_top20.shape, stock_index_top30.shape, stock_index_over1b.shape, stock_index_cutoff.shape)

# Construct Index as of Start Date

In [ ]:
stock_data_filtered.shape

In [ ]:
# QC Market Cap cutoffs:
stock_data_filtered.to_csv('Initial_MCs.csv')

In [ ]:
stock_index_top20

## Manual Adjustments (if needed)
### Adjust Methodology Here:

In [ ]:
for df in df_list:
    if 'MLMFI.PA' in df.index:
        df.drop('MLMFI.PA', axis=0, inplace=True)
        
        
'''
    
    ## Eliminate if market cap < 100M:
    
    if '1346.HK' in df.index:
        df.drop('1346.HK', axis=0, inplace=True)
    if 'BIO.AX' in df.index:
        df.drop('BIO.AX', axis=0, inplace=True)
    if 'RETI.MI' in df.index:
        df.drop('RETI.MI', axis=0, inplace=True)
        
    ## Eliminate if price < $1.00:
   
    if '2897.TW' in df.index:
        df.drop('2897.TW', axis=0, inplace=True)
    if 'KMD.NZ' in df.index:
        df.drop('KMD.NZ', axis=0, inplace=True)
    if 'VHT.AX' in df.index:
        df.drop('VHT.AX', axis=0, inplace=True)    
'''        

In [ ]:
assert stock_index_top20.shape[0] == 20
assert stock_index_top30.shape[0] == 30

# DECISION:
# Set Index and Get Price History

In [ ]:
stock_index = stock_index_top20.copy()
start_date = s
end_date = '2022-12-31'

In [ ]:
def get_price_history (df, start = start_date, end=end_date):
    tickers_list = df.index.tolist()
    tickers_data= {}
    first_object = yf.Ticker(tickers_list[0])
    
    prices = pd.DataFrame.from_dict(first_object.history(start=start, end=end, interval = "1d"))
    prices.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    prices.columns = [tickers_list[0]]

    
    for ticker in tickers_list[1:]:
        ticker_object = yf.Ticker(ticker)
        try:
            temp = pd.DataFrame.from_dict(ticker_object.history(start=start, end=end, interval = "1d"))
            temp.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
            temp.columns = [ticker]
            prices = prices.merge(temp, how="outer", left_index=True, right_index=True)
        except:
            continue
    
    
    # Collapse date across international zones:
    prices.index = pd.to_datetime(prices.index).date
      
    prices = prices.groupby(prices.index).ffill() 
    prices = prices.groupby(prices.index).bfill()
    prices = prices.groupby(prices.index).last()
    
    
    return prices

In [ ]:
price_history_df = get_price_history (stock_index)

stock_universe = list(price_history_df.columns)
print ("Number of stocks", len(stock_universe))

In [ ]:
display(price_history_df.head())


In [ ]:
def get_country_info(price_df, info_df):
    tickers_list = price_df.columns
    
    #access currency, impliedSharesOutstanding (local currency): ## HAVE TO REVIEW FOR STOCK CHANGES
    currencies = []
    implied_SO = []
    for col in price_df.columns:
        idx = np.where(info_df.index == col)
        currencies.append(info_df.iloc[idx]['currency'].values[0])
        implied_SO.append(info_df.iloc[idx]['impliedSharesOutstanding'].values[0])
        
    for i in range(len(price_df.columns)):
        col = price_df.columns[i]
        price_df[col+'_currency'] = currencies[i]
        price_df[col+'_SO'] = implied_SO[i]
    
    return price_df, currencies
     

In [ ]:
price_and_info_df, index_currencies = get_country_info(price_history_df, stock_index)

### Get historical exchange rates

In [ ]:
def get_historical_exchange (price_df, currencies):
    #currencies = list(set(currencies))
    
    querystring = {"base":"USD"}

    headers = {}
    try:
        with open('code.json') as file:
            headers = json.load(file)
    except FileNotFoundError:
        print("Couldn't load credentials")
    
    daily_rate_df = pd.DataFrame()
    
    for idx in price_df.index:
        loc = np.where(price_df.index == idx)
        url = "https://currency-conversion-and-exchange-rates.p.rapidapi.com/" + str(idx)
        response = requests.get(url, headers=headers, params=querystring)
        response = response.json()
        
        daily_df = pd.DataFrame(response)
        daily_df = daily_df[['date', 'rates']].reset_index()
        daily_df = daily_df.pivot(columns='index', index='date', values='rates')

        daily_rate_df = pd.concat([daily_rate_df, daily_df], axis=0)

        
    daily_rate_df.index = pd.to_datetime(daily_rate_df.index).date
    return daily_rate_df


# Decision:
### Run API to get all of the exchange rates? Or upload stored data?

In [ ]:
display(price_and_info_df.head())
display(price_and_info_df.tail())

In [ ]:
run_api = False

In [ ]:
# Load stored rates:
def load_rates(filename = "daily_rates_df_2022_2024_4_19.csv"):
    daily_rates_df = pd.read_csv(filename)
    daily_rates_df.set_index("Date", inplace=True)
    daily_rates_df.index = pd.to_datetime(daily_rates_df.index).date
    
    last_date = daily_rates_df.index[-1]
    return daily_rates_df, last_date

In [ ]:

df_dates, ed = load_rates()

currency_start = ed+timedelta(days=1)
currency_end = e


In [ ]:
import warnings
warnings.filterwarnings("ignore")

ext = str(dt.today().year) + "_" + str(dt.today().month) + "_" + str(dt.today().day)+ "_" + str(dt.today().hour)
filename = "daily_rates_df" + ext + ".csv"

if run_api:
    # adjust dates to only get currency data after stored file
    hist_currency_df = price_and_info_df.copy()
    hist_currency_df = hist_currency_df.loc[currency_start:,:]
    
    daily_rates_df = get_historical_exchange(hist_currency_df, index_currencies)
    display(daily_rates_df.head())
    display(daily_rates_df.tail())
    daily_rates_df.to_csv(filename)

else:
    daily_rates_df = df_dates.copy()
    

In [ ]:
daily_rates_df.head()

In [ ]:
daily_rates_df.tail()

In [ ]:
daily_price_and_info_df = price_and_info_df.join(daily_rates_df)
display(daily_price_and_info_df.head())
display(daily_price_and_info_df.tail())

In [ ]:
def convert_price_mktcap_USD (px_info_df, stock_list):
    
    for stock in stock_list:
        col_name = stock+"_currency"
        col2_name = stock+"_SO"
        curr = px_info_df[col_name][0]
        shares_out = px_info_df[col2_name][0]
        new_col = stock + "_px_USD"
        new_col2 = stock + "_MktCap_USD"
        px_info_df[new_col] = px_info_df[stock] / px_info_df[curr]
        px_info_df[new_col2] = px_info_df[new_col] * shares_out
    
    return px_info_df
    

In [ ]:
daily_price_MC_usd_all = convert_price_mktcap_USD(daily_price_and_info_df, stock_universe)

In [ ]:
#if download:
#    daily_price_MC_usd_all.to_csv('price_MC_USD.csv')
display(daily_price_MC_usd_all.head())
display(daily_price_MC_usd_all.tail())

### Refine DF and calculate PW and MW indicies:

In [ ]:
def get_df_in_USD(df_all):
    cols_to_keep = []
    for col in df_all:
        if "USD" in col:
            cols_to_keep.append(col)
    cols_to_keep.remove('USD')
    df_USD = df_all[cols_to_keep]
    
    px = []
    MC = []
    
    for col in cols_to_keep:
        if "_px" in col:
            px.append(col)
        if "_MktCap" in col:
            MC.append(col)

    df_USD_px = df_USD[px]
    df_MC_px = df_USD[MC]
    
    return df_USD, df_USD_px, df_MC_px

In [ ]:
daily_info_usd, daily_px_USD, daily_MC_USD = get_df_in_USD(daily_price_MC_usd_all)

#QC:
#daily_px_USD.to_csv('daily_px_test.csv')


In [ ]:
daily_px_USD.head()
#QC:
#daily_px_USD.to_csv('daily_px_test.csv')

In [ ]:
def calc_px_weighted_index(df, divisor = 0, show_chart = True):
    # Fill in missing days:
    pw_index = df.ffill()
    
    n, m = pw_index.shape
    if divisor == 0:
        divisor = m
    pw_index['Index'] = (pw_index.iloc[:,0:m].sum(axis=1)) / divisor

    # Remove NaN rows:
    #QC:
    #pw_index.to_csv('PW_index_test.csv')
    pw_index.dropna(inplace=True)
    #pw_index.sort_index(axis=0, level=0, inplace=True)
    
    if show_chart:
        
        fig, ax = plt.subplots(figsize = (12,6))
        ax.plot(pw_index['Index'], color = 'seagreen', linewidth=2)
        plt.title('Price Weighted Index | '+ str(m) + ' stocks')
        plt.annotate("Stocks in Index:", (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
        plt.annotate(stock_universe[0:int(m/2)], (0,0), (0, -30), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
        plt.annotate(stock_universe[int(m/2):], (0,0), (0, -40), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
        plt.show()
        
    return pw_index

def calc_MC_weighted_index(df, divisor = 1000000000, show_chart = True):
    # Fill in missing days:
    mcw_index = df.ffill()
    n, m = mcw_index.shape
    mcw_index['Index'] = (mcw_index.iloc[:,0:m].sum(axis=1)) / divisor
    
    # Remove NaN rows:
    mcw_index.dropna(inplace=True)
    
    if show_chart:
        
        fig, ax = plt.subplots(figsize = (12,6))
        ax.plot(mcw_index['Index'], color = 'royalblue', linewidth=2)
        plt.title('MktCap Weighted Index | '+ str(m) + ' stocks')
        plt.annotate("Stocks in Index:", (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
        plt.annotate(stock_universe[0:int(m/2)], (0,0), (0, -30), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
        plt.annotate(stock_universe[int(m/2):], (0,0), (0, -40), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
        plt.show()    
        
    return mcw_index

In [ ]:
price_wgt_index = calc_px_weighted_index(daily_px_USD, divisor = 1, show_chart=True)
mktcap_wgt_index = calc_MC_weighted_index (daily_MC_USD, divisor = 1000000000, show_chart=True)

display(price_wgt_index.head())
display(mktcap_wgt_index.head())


In [ ]:

if download:
    price_wgt_index.to_csv("PriceWgt_Index.csv")
    mktcap_wgt_index.to_csv("MarketCap_of_Index.csv")

In [ ]:
def combine_norm_and_benchmark(px_df, mc_df):
    
    combined_df = pd.concat([px_df['Index'],mc_df['Index']], axis=1)
    combined_df.columns = ['Price Weighted Index', "Mkt Cap Weighted Index"]
    
    # Add S&P
    spx_object = yf.Ticker("^GSPC")
    spx = pd.DataFrame.from_dict(spx_object.history(start=s, end=e, interval = "1d"))
    spx = spx[['Close']]
    spx.columns = ['SPX']
    spx.index = pd.to_datetime(spx.index).date
    
    # Add Russell 2000
    ru2000_object = yf.Ticker("^RUT")
    ru2000 = pd.DataFrame.from_dict(ru2000_object.history(start=s, end=e, interval = "1d"))
    ru2000 = ru2000[['Close']]
    ru2000.columns = ['RU2000']
    ru2000.index = pd.to_datetime(ru2000.index).date
    
    # Add DJ Sustainability
    #DJsus_object = pd.read_csv("DJSus_history.csv")
    #display(DJsus_object.head())
    #DJsus_object.index = pd.to_datetime(DJsus_object['Date'])
    #DJsus_object.drop(['Date'], axis=1, inplace=True)
    #DJsus_object.index = pd.to_datetime(DJsus_object.index).date
    #DJ_Sus = DJsus_object.copy()

    combined_df = pd.concat([combined_df, spx, ru2000], axis=1)
    
    combined_df = combined_df.ffill()
    display(combined_df.head())

    combined_df = combined_df.div(combined_df.iloc[0])
    return combined_df


In [ ]:
## Contain SPX, RU2000, DJ Sus separately for graphing

spx_object = yf.Ticker("^GSPC")
spx = pd.DataFrame.from_dict(spx_object.history(start=s, end=e, interval = "1d"))
spx = spx[['Close']]
spx.columns = ['SPX']
spx.index = pd.to_datetime(spx.index).date


ru2000_object = yf.Ticker("^RUT")
ru2000 = pd.DataFrame.from_dict(ru2000_object.history(start=s, end=e, interval = "1d"))
ru2000 = ru2000[['Close']]
ru2000.columns = ['RU2000']
ru2000.index = pd.to_datetime(ru2000.index).date

'''
DJsus_object = pd.read_csv("DJSus_history.csv")
#display(DJsus_object.head())
DJsus_object.index = pd.to_datetime(DJsus_object['Date'])
DJsus_object.drop(['Date'], axis=1, inplace=True)
DJsus_object.index = pd.to_datetime(DJsus_object.index).date
#display(DJsus_object.head())
'''

In [ ]:
index_comparison_df = combine_norm_and_benchmark(price_wgt_index, mktcap_wgt_index)

In [ ]:
index_comparison_df.head(5)

In [ ]:
index_comparison_df.sort_index(inplace=True)

## Preview and adjust dates if needed

In [ ]:
display(index_comparison_df.head(5))

In [ ]:
adjust = 0

In [ ]:
if adjust > 0:
    index_comparison_df = index_comparison_df.iloc[adjust:,:]
display(index_comparison_df.head(5))

if download:
    index_comparison_df.to_csv("Index_Comparison.csv")

### Graph Indices

In [ ]:
## Graph Separately
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize = (20,18))
ax0.plot(price_wgt_index['Index'], color = 'seagreen', linewidth=2, label = 'Price Weighted')
ax0.legend(loc=2)
ax1.plot(mktcap_wgt_index['Index'], color = 'royalblue', linewidth=2,label='Market Cap Weighted')
ax1.legend(loc=2)
ax2.plot(spx, color='darkorange', linewidth=2, label = 'SPX')
ax2.legend(loc=2)
plt.show()

In [ ]:
## Graph Together
#fig, ax = plt.subplots(figsize = (12,6))
index_comparison_df.plot(figsize = (12,6),
                         color = ['seagreen', 'royalblue', 'darkorange', 'firebrick', 'goldenrod'])

plt.show()

In [ ]:
# Export Index:
#index_comparison_df.to_csv("Index_Comparison.csv")

### Visualize Characteristics

In [ ]:
classify_all = stock_data_filtered.copy()
classify_index = stock_index.copy()
print(classify_all.shape, classify_index.shape)

In [ ]:
classify_index['country'].value_counts()

In [ ]:
classify_index

In [ ]:
def visualize_groups (df_all, df_some):
    # by sector:
    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_all['sector'].value_counts().index))
    #cmap = sns.color_palette("Blues", n_slices)
    cmap = sns.light_palette("steelblue", n_slices, reverse=True)
    
    ax.pie(df_all['sector'].value_counts(), labels=df_all['sector'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'smaller'}, colors = cmap)
    plt.title('Sectors in Public B-Corp Universe')
    plt.show()
    

    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_some['sector'].value_counts().index))
    #cmap = sns.color_palette("Blues", n_slices)
    cmap = sns.light_palette("steelblue", n_slices, reverse=True)
    
    ax.pie(df_some['sector'].value_counts(), labels=df_some['sector'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'smaller'}, colors = cmap)
    plt.title('Sectors in Top 20 Index')
    plt.show()  
    

    # by industry:
    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_all['industry'].value_counts().index))
    cmap = sns.light_palette("maroon", n_slices, reverse=True)
    
    ax.pie(df_all['industry'].value_counts(), labels=df_all['industry'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'xx-small'}, colors = cmap)
    plt.title('Industries in Public B-Corp Universe')
    plt.show()
    


    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_some['industry'].value_counts().index))
    cmap = sns.light_palette("maroon", n_slices, reverse=True)
    
    ax.pie(df_some['industry'].value_counts(), labels=df_some['industry'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'xx-small'}, colors = cmap)
    plt.title('Industries in Top 20 Index')
    plt.show()   
  
    
    # by country:
    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_all['country'].value_counts().index))
    #cmap = sns.color_palette("Blues", n_slices)
    cmap = sns.light_palette("seagreen", n_slices, reverse=True)
    
    ax.pie(df_all['country'].value_counts(), labels=df_all['country'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'smaller'}, colors = cmap)
    plt.title('Countries in Public B-Corp Universe')
    plt.show()
    
    # by country:
    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_some['country'].value_counts().index))
    #cmap = sns.color_palette("Blues", n_slices)
    cmap = sns.light_palette("seagreen", n_slices, reverse=True)
    
    ax.pie(df_some['country'].value_counts(), labels=df_some['country'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'smaller'}, colors = cmap)
    plt.title('Countries in Top 20 Index')
    plt.show()


    
    # by currency:
    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_all['currency'].value_counts().index))
    #cmap = sns.color_palette("Blues", n_slices)
    cmap = sns.light_palette("goldenrod", n_slices, reverse=True)
    
    ax.pie(df_all['currency'].value_counts(), labels=df_all['currency'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'smaller'}, colors = cmap)
    plt.title('Currencies in Public B-Corp Universe')
    plt.show()
    
    # by currency:
    fig, ax = plt.subplots(figsize = (8,6))
    
    n_slices = len(list(df_some['currency'].value_counts().index))
    #cmap = sns.color_palette("Blues", n_slices)
    cmap = sns.light_palette("goldenrod", n_slices, reverse=True)
    
    ax.pie(df_some['currency'].value_counts(), labels=df_some['currency'].value_counts().index, autopct='%1.1f%%',
          pctdistance=.8, textprops={'size': 'smaller'}, colors = cmap)
    plt.title('Currencies in Top 20 Index')
    plt.show()

In [ ]:
visualize_groups(classify_all, classify_index)

In [ ]:
print(stock_index.shape)

In [ ]:
print(price_wgt_index.shape)
display(price_wgt_index)

In [ ]:
print(mktcap_wgt_index.shape)
display(mktcap_wgt_index)

## Reconstitute:

In [ ]:
df22 = stock_data_filtered.copy()

In [ ]:
def reconstitute(df, recon_date):

    # Confirm Implied Shares Outstanding for Mkt Cap calculation:    
    df['impliedSharesOutstanding'] = df['marketCap'] / df['previousClose']
    
    reprice = get_starting_price(df, start = recon_date)
    
    recon_px = reprice.filter(like=recon_date, axis=0).T
    recon_px.columns = [str(recon_date)]
    
    exch_data_recast = get_rates(date=recon_date)
    currency_dict_recast = currency_dict.copy()
    
    # Revise currency translation:
    for symbol, rate in exch_data_recast['rates'].items():
        for key in currency_dict_recast.keys():
            if symbol == key:
                currency_dict_recast[key][1] = rate
                
    # Revise currency map:
    currency_map_rev = {symbol:rate
                       for symbol, name in currency_dict_recast.items()
                       for rate in name}
    
    col = 'USDExchRate'+str(recon_date)
    df[col] = df['currency'].map(currency_map_rev)
    df = df.join(recon_px)
    
    
    col2 = 'CloseUSD'+str(recon_date)
    df[col2] = df[recon_date] / df[col]
    
    col3='MarketCapUSD'+str(recon_date)
    df[col3] = df[col2] * df['impliedSharesOutstanding']
    
    
    return df

In [ ]:
df23 = reconstitute(df22, recon_date = '2023-01-03') 
display(df23.head(20))

df24 = reconstitute(df23, recon_date = '2024-01-02') 
display(df24.head(20))

In [ ]:
df23.shape

In [ ]:
#df24.to_csv('reconst_test.csv')

In [ ]:
def construct_index_with_recon (df, cutoff=20, start_date = '2022-01-04', end_date = '2022-12-31', pw_divisor = 1, mw_divisor=1000000000):
    today = str(dt.today())
    if today >'2024-12-31':
        print("Add a Year!")
    
    # Clean up errant stock:
    if 'MLMFI.PA' in df.index:
        df.drop('MLMFI.PA', axis=0, inplace=True)

    
    if str(start_date[0:4]) == "2022":
        col = 'beginningMarketCapUSD'
    else:
        col = 'MarketCapUSD' + start_date
    index = filter_index(df, size=False, num=True, num_thresh=cutoff, col = col)
    
    #display(index.head())
    
    assert index.shape[0] == cutoff
    
    prices = get_price_history(index, start_date, end= end_date)
    price_and_info, index_currencies = get_country_info(prices, index)
    
    
    # Set stock universe
    stock_universe = list(index.T.columns)
    
    #Get historical exchange rates
    filename = "daily_rates_" + str(start_date[0:4] + ".csv")
    daily_rates_df, _ = load_rates(filename)

    daily_price_and_info_df = price_and_info.join(daily_rates_df)
    
    # Add marketcap:
    daily_price_MC_usd_all = convert_price_mktcap_USD(daily_price_and_info_df, stock_universe) 
    
    # Refine DF INTO USD and calculate PW and MW indicies:
    daily_info_usd, daily_px_USD, daily_MC_USD = get_df_in_USD(daily_price_MC_usd_all)
    
  
    price_wgt_index = calc_px_weighted_index(daily_px_USD, divisor = pw_divisor, show_chart=False)
    
    #display(price_wgt_index.head())
    #display(price_wgt_index.tail())
    
    
    mktcap_wgt_index = calc_MC_weighted_index (daily_MC_USD, divisor = mw_divisor, show_chart=False)
    

    #filename2 = 'px_weighted_idx_details'+ str(start_date[0:4]) + ".csv"
    #price_wgt_index.to_csv(filename2)

    ending_index_pw = price_wgt_index.iloc[-1,-1]
    ending_index_mkt = mktcap_wgt_index.iloc[-1,-1]
    
    return price_wgt_index, ending_index_pw, mktcap_wgt_index, ending_index_mkt


In [ ]:
def set_divisor (df0, df1, s0 = '2022-01-04', e0 = '2022-12-31', s1 = '2023-01-03', e1='2023-12-31', divisor=1):
    _, end0_pw, _, end0_mkt = construct_index_with_recon(df0, start_date = s0, end_date = e0, pw_divisor = divisor, mw_divisor = divisor)
    print("end year 0", end0_pw)
    px_wgt_1, _, mkt_cap1, _ = construct_index_with_recon(df1, start_date = s1, end_date = e1, pw_divisor = divisor, mw_divisor = divisor)
        
    
    beg1_pw = px_wgt_1.iloc[0,-1]
    print("beg year 1", beg1_pw)
    
    divisor1_pw = divisor * (beg1_pw/end0_pw)
    print("PW Divisor = ", divisor1_pw)
    
    beg1_mkt = mkt_cap1.iloc[0,-1]
    print("beg year 1", beg1_mkt)
    
    divisor1_mkt = divisor * (beg1_mkt/end0_mkt)
    print("Mkt Divisor = ", divisor1_mkt)
    
    return divisor1_pw, divisor1_mkt
    

### COMPILE PX WEIGHTED INDEX and GRAPH PERFORMANCE

In [ ]:
price_wgt_index_22, _, _, _ = construct_index_with_recon(df22, pw_divisor = 1)
div1, _ = set_divisor(df22, df23, s0 = '2022-01-04', e0 = '2022-12-31', s1 = '2023-01-03', e1='2023-12-31', divisor=1)

price_wgt_index_23, _, _, _ = construct_index_with_recon(df23, start_date = '2023-01-03', end_date = '2023-12-31', pw_divisor = div1)
div2, _ = set_divisor(df23, df24, s0 = '2023-01-03', e0 = '2023-12-31', s1 = '2024-01-02', e1='2024-04-15', divisor = div1)

price_wgt_index_24, _, _, _ = construct_index_with_recon(df24, start_date = '2024-01-02', end_date = '2024-04-15', pw_divisor = div2)



In [ ]:
index_df = pd.concat([price_wgt_index_22.iloc[:,-1], price_wgt_index_23.iloc[:,-1]])
index_df = pd.concat([index_df, price_wgt_index_24.iloc[:,-1]])

#index_df

m = 20
fig, ax = plt.subplots(figsize = (12,6))
ax.plot(index_df, color = 'seagreen', linewidth=2)
plt.title('Price Weighted Index | '+ str(m) + ' stocks')
#plt.annotate("Stocks in Index:", (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
#plt.annotate(stock_universe[0:int(m/2)], (0,0), (0, -30), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
#plt.annotate(stock_universe[int(m/2):], (0,0), (0, -40), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
plt.show()


In [ ]:
dfs = [df22, df23, df24]
px_indexs = [price_wgt_index_22, price_wgt_index_23, price_wgt_index_24]

for i in range(len(dfs)):
    tickers = [col[:-7] for col in px_indexs[i].columns][:-1]
    info = dfs[i].loc[dfs[i].index.isin(tickers)]

    visualize_groups(classify_all, info)


### COMPILE Market WEIGHTED INDEX and GRAPH PERFORMANCE

In [ ]:
_,_,mkt_wgt_index_22,_ = construct_index_with_recon(df22, mw_divisor = 1000000000)
_, div1 = set_divisor(df22, df23, s0 = '2022-01-04', e0 = '2022-12-31', s1 = '2023-01-03', e1='2023-12-31', divisor=1000000000)

_, _, mkt_wgt_index_23, _ = construct_index_with_recon(df23, start_date = '2023-01-03', end_date = '2023-12-31', pw_divisor = div1)
_, div2 = set_divisor(df23, df24, s0 = '2023-01-03', e0 = '2023-12-31', s1 = '2024-01-02', e1='2024-04-15', divisor = div1)

_, _, mkt_wgt_index_24, _ = construct_index_with_recon(df24, start_date = '2024-01-02', end_date = '2024-04-15', pw_divisor = div2)



In [ ]:
mkt_index_df = pd.concat([mkt_wgt_index_22.iloc[:,-1], mkt_wgt_index_23.iloc[:,-1]])
mkt_index_df = pd.concat([mkt_index_df, mkt_wgt_index_24.iloc[:,-1]])

#index_df

m = 20
fig, ax = plt.subplots(figsize = (12,6))
ax.plot(mkt_index_df, color = 'royalblue', linewidth=2)
plt.title('Market Weighted Index | '+ str(m) + ' stocks')
#plt.annotate("Stocks in Index:", (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
#plt.annotate(stock_universe[0:int(m/2)], (0,0), (0, -30), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
#plt.annotate(stock_universe[int(m/2):], (0,0), (0, -40), xycoords='axes fraction', textcoords='offset points', va='top', color = '#666666', fontsize=9)
plt.show()


In [ ]:
if download:
    price_wgt_index_22.to_csv("px_weighted_idx_details_2022.csv")
    price_wgt_index_23.to_csv("px_weighted_idx_details_2023.csv")
    price_wgt_index_24.to_csv("px_weighted_idx_details_2024.csv")
    
    mkt_wgt_index_22.to_csv("mkt_weighted_idx_details_2022.csv")
    mkt_wgt_index_23.to_csv("mkt_weighted_idx_details_2023.csv")
    mkt_wgt_index_24.to_csv("mkt_weighted_idx_details_2024.csv")
        

Attributes = ['address1',
 'city',
 'state',
 'zip',
 'country',
 'phone',
 'website',
 'industry',
 'industryKey',
 'industryDisp',
 'sector',
 'sectorKey',
 'sectorDisp',
 'longBusinessSummary',
 'fullTimeEmployees',
 'companyOfficers',
 'auditRisk',
 'boardRisk',
 'compensationRisk',
 'shareHolderRightsRisk',
 'overallRisk',
 'governanceEpochDate',
 'compensationAsOfEpochDate',
 'maxAge',
 'priceHint',
 'previousClose',
 'open',
 'dayLow',
 'dayHigh',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'regularMarketDayLow',
 'regularMarketDayHigh',
 'beta',
 'forwardPE',
 'volume',
 'regularMarketVolume',
 'averageVolume',
 'averageVolume10days',
 'averageDailyVolume10Day',
 'bid',
 'ask',
 'bidSize',
 'askSize',
 'marketCap',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'priceToSalesTrailing12Months',
 'fiftyDayAverage',
 'twoHundredDayAverage',
 'currency',
 'enterpriseValue',
 'profitMargins',
 'floatShares',
 'sharesOutstanding',
 'sharesShort',
 'sharesShortPriorMonth',
 'sharesShortPreviousMonthDate',
 'dateShortInterest',
 'sharesPercentSharesOut',
 'heldPercentInsiders',
 'heldPercentInstitutions',
 'shortRatio',
 'shortPercentOfFloat',
 'impliedSharesOutstanding',
 'bookValue',
 'priceToBook',
 'lastFiscalYearEnd',
 'nextFiscalYearEnd',
 'mostRecentQuarter',
 'netIncomeToCommon',
 'trailingEps',
 'forwardEps',
 'pegRatio',
 'enterpriseToRevenue',
 'enterpriseToEbitda',
 '52WeekChange',
 'SandP52WeekChange',
 'exchange',
 'quoteType',
 'symbol',
 'underlyingSymbol',
 'shortName',
 'longName',
 'firstTradeDateEpochUtc',
 'timeZoneFullName',
 'timeZoneShortName',
 'uuid',
 'messageBoardId',
 'gmtOffSetMilliseconds',
 'currentPrice',
 'targetHighPrice',
 'targetLowPrice',
 'targetMeanPrice',
 'targetMedianPrice',
 'recommendationMean',
 'recommendationKey',
 'numberOfAnalystOpinions',
 'totalCash',
 'totalCashPerShare',
 'ebitda',
 'totalDebt',
 'quickRatio',
 'currentRatio',
 'totalRevenue',
 'debtToEquity',
 'revenuePerShare',
 'returnOnAssets',
 'returnOnEquity',
 'freeCashflow',
 'operatingCashflow',
 'revenueGrowth',
 'grossMargins',
 'ebitdaMargins',
 'operatingMargins',
 'financialCurrency',
 'trailingPegRatio']